In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sklearn.metrics
import math

In [7]:
#Q1
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_jan.shape

(3066766, 19)

In [8]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [9]:
#Q2
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
print(np.std(df_jan.duration))

42.59434429744777


In [10]:
#Q3
df_jan.shape

(3066766, 20)

In [11]:
df_jan_no_outliers = df_jan.loc[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]
df_jan_no_outliers.shape

(3009173, 20)

In [12]:
3009173 / 3066766 * 100

98.1220282212598

In [13]:
#Q4
df_encoded = pd.get_dummies(df_jan_no_outliers[['PULocationID', 'DOLocationID']], 
                            columns = ['PULocationID', 'DOLocationID'], 
                            dtype=int)
df_encoded.head()

,PULocationID_1,PULocationID_2,PULocationID_3,PULocationID_4,PULocationID_5,PULocationID_6,PULocationID_7,PULocationID_8,PULocationID_9,PULocationID_10,...,DOLocationID_256,DOLocationID_257,DOLocationID_258,DOLocationID_259,DOLocationID_260,DOLocationID_261,DOLocationID_262,DOLocationID_263,DOLocationID_264,DOLocationID_265
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Q5
x = df_encoded.to_numpy()
y = df_jan_no_outliers['duration'].to_numpy()

In [15]:
model = LinearRegression().fit(x, y)

In [16]:
y_pred = model.predict(x)

In [17]:
rmse = math.sqrt(sklearn.metrics.mean_squared_error(y, y_pred))
print(rmse)

7.6492642338618495


In [18]:
#Q6
# compute duration 
df_feb['duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

# remove outliers
df_feb_no_outliers = df_feb.loc[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]

#one hot encoding
df_feb_encoded = pd.get_dummies(df_feb_no_outliers[['PULocationID', 'DOLocationID']], 
                            columns = ['PULocationID', 'DOLocationID'], 
                            dtype=int)
x_feb = df_feb_encoded.to_numpy()
y_feb = df_feb_no_outliers['duration'].to_numpy()

# train model
model = LinearRegression().fit(x_feb, y_feb)

# calculate RMSE 
y_feb_pred = model.predict(x_feb)
feb_rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_feb, y_feb_pred))
print(feb_rmse)

7.77895573802682
